In [ ]:
%run ../Helper/Setup.ipynb

In [ ]:
def import_csv(file, group=True):
    df = pd.read_csv(file)
    
    if group:
        df = group_data(df)
    
    default_buffer = 212992
    df['udp_buffersize_absolute'] = df.apply(lambda x: int(x.logdir.split('-')[-1]), axis=1)
    df['udp_buffersize'] = df['udp_buffersize_absolute'] / 212992
    
    df = df[~np.isnan(df.goodput)]
    
    return df

def group_data(df):
    filtered = df.groupby(by=['server', 'client'], as_index=False).agg({'goodput': ['mean', 'std'], 'repetition': 'count'})
    filtered.columns = list(map(''.join, filtered.columns.values))
    return filtered

In [ ]:
udp_buffer = import_csv('../../results/buffer/udp/processed.csv', group=False)

In [ ]:
def plot_goodput(df):

    implementations = pd.unique(df[['server', 'client']].values.ravel('K'))
    label = sorted(df.udp_buffersize.unique())
    print(label)
    
    implementation_order = {
        'lsquic': 1,
        'quiche': 2,
        'tcp-tls': 3
    }
    
    implementations = sorted(implementations, key=lambda x: implementation_order.get(x, 1000))
    
    implementation_names = {
        'lsquic': 'LSQUIC',
        'quiche': 'quiche',
        'quich': 'quiche',
        'tcp+tls': 'TCP/TLS',
    }

        
    colors = [ColorDarkerBlue, ColorGreen, ColorRed]
    patterns = [" "] * len(colors)
    linestyles = ['-'] * len(colors)
    markers = ['^', 'o', 's']
    
    def style(i):
        return {
            #"boxprops": dict(facecolor=c, color=c),
            "showmeans": True,
            "boxprops": dict(color=colors[i], linestyle=linestyles[i], facecolor="white"),
            "capprops": dict(color=colors[i], alpha=0),
            "whiskerprops": dict(alpha=0),
            "flierprops": dict(color=colors[i], markeredgecolor=colors[i]),
            #"medianprops": dict(color=c),
            #"meanprops": dict(color=c),
            "medianprops": dict(color=colors[i]),
            "meanprops": dict(markerfacecolor=colors[i], markeredgecolor=colors[i], marker=markers[i]),
            "whis": [5,95],
            "showfliers": False,
            "widths": .22,
            "positions": [x - 0.15 + 0.3 * i for x in range(len(label))],
            "patch_artist": True,
        }
    
    width, height = set_size('single')
    
    fig, ax1 = plt.subplots(figsize=(width, height*0.8))
    plot_shades(label, ax1, 0.5)
    
    for idx, imp in enumerate(implementations):
        
        d = df.loc[(df.server == imp) & (df.client == imp)]
        bp1 = ax1.boxplot([d[d.udp_buffersize == c].goodput for c in label], labels=label,
            #hatch=patterns[0],
            **style(idx),
           )
    
    ax1.autoscale()
    
    ax1.set(xticks=range(len(label)), xticklabels=label)
    
    ax1.set_xlabel('UDP receive buffer size in multiples of default size')
    ax1.set_ylabel('Goodput [Gbit/s]')
    ax1.set_ylim(0, 3800)
    #ax1.set_xscale('log')
    
    mkfunc = lambda x, pos: f'{(x * 1e-3):.0f}'
    mkformatter = matplotlib.ticker.FuncFormatter(mkfunc)
    ax1.yaxis.set_major_formatter(mkformatter)

    
    legend_elements = [
        mlines.Line2D([], [], color=colors[i], marker=markers[i], linestyle='None', label=implementation_names[imp])
        for i, imp in enumerate(implementations)
    ]
    
    
    ax1.legend(handles=legend_elements, loc="lower right", facecolor='white', framealpha=1, ncol=1)
    fig.tight_layout()
    fig.savefig('udp-buffer.pdf')


In [ ]:
plot_goodput(udp_buffer)

In [ ]:
def plot_packets_sent(df):

    implementations = pd.unique(df[['server', 'client']].values.ravel('K'))
    label = sorted(df.udp_buffersize.unique())
    print(label)
    
    implementation_order = {
        'lsquic': 1,
        'quiche': 2,
        'tcp-tls': 3
    }
    
    implementations = sorted(implementations, key=lambda x: implementation_order.get(x, 1000))
    
    implementation_names = {
        'lsquic': 'LSQUIC',
        'quiche': 'quiche',
        'quich': 'quiche',
        'tcp+tls': 'TCP/TLS',
    }

        
    colors = [ColorDarkerBlue, ColorGreen, ColorRed]
    patterns = [" "] * len(colors)
    linestyles = ['-'] * len(colors)
    markers = ['^', 'o', 's']
    
    def style(i):
        return {
            #"boxprops": dict(facecolor=c, color=c),
            "showmeans": True,
            "boxprops": dict(color=colors[i], linestyle=linestyles[i], facecolor="white"),
            "capprops": dict(color=colors[i], alpha=0),
            "whiskerprops": dict(alpha=0),
            "flierprops": dict(color=colors[i], markeredgecolor=colors[i]),
            #"medianprops": dict(color=c),
            #"meanprops": dict(color=c),
            "medianprops": dict(color=colors[i]),
            "meanprops": dict(markerfacecolor=colors[i], markeredgecolor=colors[i], marker=markers[i]),
            "whis": [5,95],
            "showfliers": False,
            "widths": .22,
            "positions": [x - 0.15 + 0.3 * i for x in range(len(label))],
            "patch_artist": True,
        }
    
    
    width, height = set_size('single')
    
    fig, ax1 = plt.subplots(figsize=(width, height*0.8))
    plot_shades(label, ax1, 0.5)

    for idx, imp in enumerate(implementations):
        
        d = df.loc[(df.server == imp) & (df.client == imp)]
        bp1 = ax1.boxplot([d[d.udp_buffersize == c].client_ethtool_tx_total_frames for c in label], labels=label,
            #hatch=patterns[0],
            **style(idx),
           )
    
    ax1.autoscale()

    ax1.set(xticks=range(len(label)), xticklabels=label)
    
    ax1.set_xlabel('UDP receive buffer size in multiples of default size')
    ax1.set_ylabel('\# Packets sent by client')
    ax1.set_ylim(0, 570000)
    #ax1.set_xscale('log')
    
    mkfunc = lambda x, pos: f'{(x * 1e-3):.0f}k'
    mkformatter = matplotlib.ticker.FuncFormatter(mkfunc)
    ax1.yaxis.set_major_formatter(mkformatter)
    
    legend_elements = [
        mlines.Line2D([], [], color=colors[i], marker=markers[i], linestyle='None', label=implementation_names[imp])
        for i, imp in enumerate(implementations)
    ]
    
    
    ax1.legend(handles=legend_elements, loc="lower right", facecolor='white', framealpha=1, ncol=1,
              bbox_to_anchor=(1, 0.15),
              )
    fig.tight_layout()
    fig.savefig('udp-buffer-number-acks.pdf')


In [ ]:
plot_packets_sent(udp_buffer)